## Import Libraries

In [ ]:
import pandas as pd
import os
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

## Read Config File

In [ ]:
import configparser 
config = configparser.ConfigParser()
config.read('config.ini')
ip = config['DEFAULT']['IP']
port = config['DEFAULT']['MongoDB-Port']
output_folder = config['DEFAULT']['Output-Folder']
db_name = config['DEFAULT']['DB-Name']
user_name = config['DEFAULT']['User-Name']
psword = config['DEFAULT']['Psword']

## Connect MOngoDB

In [ ]:
from pymongo import MongoClient
client = MongoClient(ip, int(port), username=user_name, password=psword)

## Get Collection Names

In [ ]:
db = client[db_name]
collections = db.collection_names()

## Supporting Functions

In [ ]:
# create foler if not exist
def create_folder(folder):
    if not os.path.exists(folder):
        os.makedirs(folder)

## Export CSV from MongoDB

In [ ]:
create_folder(output_folder)
for c in collections:
    df = pd.DataFrame()
    cursor = db[c].find({})
    
    # Expand the cursor and construct the DataFrame
    for i in list(cursor):
        df = df.append(i,ignore_index=True)
    
    # Delete the _id
    if '_id' in df:
        del df['_id']
        
    # write into csv
    df.to_csv("{}{}.csv".format(output_folder,c),sep=',',index=False,encoding ="UTF-8")